In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
# usual imports
import os
import sys
import numpy as np
from rail.core.utils import RAILDIR
#from rail.estimation.algos.sompz_version.utils import RAIL_SOMPZ_DIR
#from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params
import matplotlib.pyplot as plt

import pandas as pd
import astropy.io.fits as fits

In [ ]:
# use local repo with local edits
# rail_sompz_repo_develop_list = ["/Users/jmyles/repositories/rail_sompz",
#                                 "/Users/jmyles/repositories/rail_sompz/lib",
#                                 "/Users/jmyles/repositories/rail_sompz/lib/python3.10",
#                                 "/Users/jmyles/repositories/rail_sompz/lib/python3.10/site-packages"]
# for rail_sompz_repo_develop in rail_sompz_repo_develop_list:
#     sys.path.insert(0, rail_sompz_repo_develop)
from rail.estimation.algos.sompz import SOMPZInformer
from rail.estimation.algos.sompz import SOMPZEstimator

In [ ]:
from rail.core.data import TableHandle
from rail.core.stage import RailStage

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
from rail.core.utils import find_rail_file
#trainFile = find_rail_file('examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')
#deep_data = DS.read_file("training_data", TableHandle, trainFile)
#wide_data = DS.read_file("input_wide_data", TableHandle, testFile)

In [ ]:
#datadir = '/global/cfs/projectdirs/desc-pz/users/jmyles/sompz_desc/' 
datadir = '/Users/jmyles/data/sompz_desc'
hsc_file = os.path.join(datadir, "hsc_deep_incl_nir_incl_specz_TEST_ONLY_NO_SCIENCE_2023-10-27_small.fits")
if not os.path.exists(hsc_file):
    data = fits.open(hsc_file.replace('_small',''))[1].data
    fits.writeto(hsc_file, data[:5000])

In [ ]:
spec_data = DS.read_file("training_data", TableHandle, hsc_file)
balrog_data = DS.read_file("training_data", TableHandle, hsc_file)
wide_data = DS.read_file("training_data", TableHandle, hsc_file)

In [ ]:
bands = ['u','g','r','i','z','y','J','H', 'K']

deepbands = []
deeperrs = []
zeropts = []
for band in bands:
    deepbands.append(f'flux_DEEP_{band}')
    deeperrs.append(f'flux_err_DEEP_{band}')
    zeropts.append(30.)

widebands = []
wideerrs = []  
for band in bands[1:5]:
    widebands.append(f'{band}_cmodel_flux')
    wideerrs.append(f'{band}_cmodel_fluxerr')
    
refband_deep=deepbands[3]
refband_wide=widebands[2]

In [ ]:
som_params = dict(inputs_deep=deepbands, input_errs_deep=deeperrs, 
                  zero_points_deep=zeropts, 
                  convert_to_flux_deep=False, convert_to_flux_wide=False, 
                  set_threshold_deep=True, thresh_val_deep=1.e-5, 
                  som_shape_wide=(25, 25), som_minerror_wide=0.005,
                  som_take_log_wide=False, som_wrap_wide=False)

In [ ]:
som_estimate = SOMPZEstimator.make_stage(name="som_estimator", 
                                         data_path=testFile,
                                         deep_bands=deepbands,
                                         err_deep_bands=deeperrs,  
                                         ref_band_deep=refband_deep,
                                         wide_bands=widebands,
                                         err_wide_bands=wideerrs,    
                                         ref_band_wide=refband_wide,
                                         deep_groupname='', 
                                         hdf5_groupname='',
                                         model="TEST_HSC_model_2023-10-27b.pkl", 
                                         **som_params)

In [ ]:
som_estimate.estimate(spec_data,
                      balrog_data,
                      wide_data)

In [ ]:
som_estimate